<a href="https://colab.research.google.com/github/Sarithalakshman/Assignment_1/blob/main/clientQMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import mysql.connector
import hashlib
from datetime import datetime

# Database connection function
def get_connection():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="srihari",
        database="client_support"
    )
    return conn


In [ ]:
def register_user(username, password, role):
    conn = get_connection()
    cur = conn.cursor()

    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    sql = "INSERT INTO users (username, hashed_password, role) VALUES (%s, %s, %s)"
    cur.execute(sql, (username, hashed_password, role))

    conn.commit()
    conn.close()
    print("✅ User registered successfully!")


In [ ]:
def login_user(username, password):
    conn = get_connection()
    cur = conn.cursor()

    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    sql = "SELECT role FROM users WHERE username=%s AND hashed_password=%s"
    cur.execute(sql, (username, hashed_password))
    row = cur.fetchone()

    conn.close()

    if row:
        print(f" Login successfull Role: {row[0]}")
        return row[0]
    else:
        print(" Invalid username or password.")
        return None


In [ ]:
def submit_query(query_id, email, mobile, heading, description):
    conn = get_connection()
    cur = conn.cursor()

    date_raised = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    status = "Open"
    date_closed = None

    sql = """INSERT INTO client_queries
            (query_id, client_email, client_mobile, query_heading, query_description, status, date_raised, date_closed)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""

    cur.execute(sql, (query_id, email, mobile, heading, description, status, date_raised, date_closed))
    conn.commit()
    conn.close()
    print(" Query submitted successfully!")


In [ ]:
def view_queries(status=None):
    conn = get_connection()
    cur = conn.cursor()

    if status:
        cur.execute("SELECT * FROM client_queries WHERE status=%s", (status,))
    else:
        cur.execute("SELECT * FROM client_queries")

    rows = cur.fetchall()
    conn.close()

    for row in rows:
        print(row)


In [ ]:
def close_query(query_id):
    conn = get_connection()
    cur = conn.cursor()

    date_closed = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sql = "UPDATE client_queries SET status=%s, date_closed=%s WHERE query_id=%s AND status='Open'"
    cur.execute(sql, ("Closed", date_closed, query_id))

    if cur.rowcount == 0:
        print("Query not found or already closed.")
    else:
        conn.commit()
        print("Query closed successfully.")

    conn.close()


In [ ]:
# Register some users
register_user("sriharan", "password3", "Client")
register_user("support4", "admin3", "Support")

# Login as client
role = login_user("sriharan", "password3")

# Client submits query
if role == "Client":
    submit_query("Q007", "sriharan@gmail.com", "9876543211", "Computer is Slow", "My computer is running very slowly.")

# Login as support
role = login_user("support4", "admin3")

# Support views and closes query
if role == "Support":
    print("📋 Open Queries:")
    view_queries("Open")

    close_query("Q007")
    print("📋 After closing:")
    view_queries()

✅ User registered successfully!
✅ User registered successfully!
 Login successfull Role: Client
✅ Query submitted successfully!
 Login successfull Role: Support
📋 Open Queries:
('Q003', 'gopal@gmail.com', '9876465778', 'bad networking', "i can't open it.", 'Open', datetime.datetime(2025, 9, 18, 10, 38, 42), None, None)
('Q006', 'senthil@gmail.com', '944734240', 'Printer Not Working', 'The office printer is not responding.', 'Open', datetime.datetime(2025, 9, 21, 8, 38, 58), None, None)
('Q007', 'sriharan@gmail.com', '9876543211', 'Computer is Slow', 'My computer is running very slowly.', 'Open', datetime.datetime(2025, 9, 22, 15, 52, 31), None, None)
Query closed successfully.
📋 After closing:
('Q001', 'john@email.com', '9876543210', 'Login Issue', 'I cannot log in.', 'Closed', datetime.datetime(2025, 9, 15, 17, 17, 5), datetime.datetime(2025, 9, 15, 17, 17, 7), None)
('Q003', 'gopal@gmail.com', '9876465778', 'bad networking', "i can't open it.", 'Open', datetime.datetime(2025, 9, 18, 